**News Data**

In [1]:
import requests
import pandas as pd
import numpy as np

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

In [2]:
BASE_URL = "https://content.guardianapis.com/search"
def news_api(API_KEY: str):
    params = {
        "api-key": API_KEY,
        "from-date": "2025-03-02",
        "to-date": "2025-03-02",
        "sectionId": "business",
        # "sectionName": "business",
        "q": "US stock market AND Wall street AND US Economy AND US finance AND American business",
        "show-fields": "headline, body",
        "order-by": "newest",
        "page-size": 100
    }

    response = requests.get(BASE_URL, params)
    data = response.json()

    articles = []
    for article in data["response"]["results"]:
        articles.append({
            "Title": article["webTitle"],
            "URL": article["webUrl"],
            "Publication Date": article["webPublicationDate"],
        })

    df = pd.DataFrame(articles)
    df = df.set_index("Publication Date")

    return df

In [3]:
news_data = news_api("75e3c8c0-28e6-4166-961c-a72883c8ea3a")

In [4]:
news_data

,Title,URL
Publication Date,,
2025-03-02T23:34:24Z,Greek PM vows to upgrade railways as government faces confidence vote,https://www.theguardian.com/world/2025/mar/02/greek-pm-vows-upgrade-railways-government-faces-confidence-vote
2025-03-02T23:11:18Z,Multiple wildfires in North and South Carolina force evacuations,https://www.theguardian.com/us-news/2025/mar/02/wildfires-north-south-carolina-evacuations
2025-03-02T22:37:22Z,Keir Starmer says Europe ‘at crossroads in history’ and must support Ukraine,https://www.theguardian.com/world/2025/mar/02/keir-starmer-europe-crossroads-history-must-support-ukraine
2025-03-02T21:55:33Z,Defiant but tactful Zelenskyy seeks to move on from White House fiasco,https://www.theguardian.com/world/2025/mar/02/zelenskyy-offers-no-apology-to-trump-as-he-says-row-brought-nothing-positive
2025-03-02T21:39:25Z,Trump invites freed Israeli hostage to White House,https://www.theguardian.com/us-news/2025/mar/02/trump-israeli-hostage-eli-sharabi
2025-03-02T21:36:39Z,Geyse ‘lonely’ at Manchester United as Gotham FC enter loan talks,https://www.theguardian.com/football/2025/mar/02/geyse-describes-lonely-feeling-amid-manchester-united-loan-allegations
2025-03-02T21:14:15Z,Fulham’s sensible plan punishes a fading institution with stubborn tactics | John Brewin,https://www.theguardian.com/football/2025/mar/02/fulhams-sensible-plan-punishes-fading-institution-with-ruben-amorims-stubborn-tactics
2025-03-02T21:14:12Z,Amorim hits back at Rooney after Manchester United cup exit to Fulham,https://www.theguardian.com/football/2025/mar/02/ruben-amorim-wayne-rooney-manchester-united-fulham-fa-cup
2025-03-02T21:00:17Z,Corrections and clarifications,https://www.theguardian.com/news/2025/mar/02/corrections-and-clarifications1


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import scipy
import tqdm as notebook_tqdm

C:\ProgramData\anaconda3\envs\RL_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def calculate_compound_scores(probabilities):
    return probabilities['positive'] - probabilities['negative']

In [7]:
news_titles = list(news_data["Title"].values)
print(news_titles)

['Greek PM vows to upgrade railways as government faces confidence vote', 'Multiple wildfires in North and South Carolina force evacuations', 'Keir Starmer says Europe ‘at crossroads in history’ and must support Ukraine', 'Defiant but tactful Zelenskyy seeks to move on from White House fiasco', 'Trump invites freed Israeli hostage to White House', 'Geyse ‘lonely’ at Manchester United as Gotham FC enter loan talks', 'Fulham’s sensible plan punishes a fading institution with stubborn tactics | John Brewin', 'Amorim hits back at Rooney after Manchester United cup exit to Fulham', 'Corrections and clarifications', 'Starmer struggles to remain upright under the weight of his contradictions | Zoe Williams', 'Morning Mail: AFL loses control on gambling oversight; Europe at Ukraine ‘crossroads’; WA renewables stall', 'Starmer’s plan for peace in Ukraine: key takeaways from London summit', 'Women’s Super League clubs will hold vote on radical plan to scrap relegation', 'Manchester United 1-1 Fu

In [8]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

inputs = tokenizer(news_titles, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    logits = model(**inputs).logits

probabilities = scipy.special.softmax(logits.numpy(), axis=1)
labels = list(model.config.id2label.values())

sentiment_scores = []
for i, title in enumerate(news_titles):
    scores_dict = {labels[j]: probabilities[i][j] for j in range(len(labels))}
    sentiment_scores.append(scores_dict)

compound_scores = [calculate_compound_scores(scores) for scores in sentiment_scores]
print(compound_scores)

[np.float32(0.85585856), np.float32(-0.92134804), np.float32(0.5051684), np.float32(-0.2585921), np.float32(0.02710332), np.float32(0.025343686), np.float32(-0.4260723), np.float32(0.6375139), np.float32(-0.027394693), np.float32(-0.060982056), np.float32(-0.8650409), np.float32(0.058208108), np.float32(0.021733552), np.float32(-0.023927208), np.float32(-0.0038070604), np.float32(0.073991194), np.float32(0.014752053), np.float32(0.15395576), np.float32(-0.022308543), np.float32(-0.020281158), np.float32(0.233286), np.float32(-0.8916016), np.float32(0.015455309), np.float32(0.14792088), np.float32(-0.7310732), np.float32(-0.9556407), np.float32(-0.0052545965), np.float32(0.04224082), np.float32(0.0050233454), np.float32(-0.019464374), np.float32(0.035365492), np.float32(0.7551421), np.float32(-0.82776386), np.float32(-0.17578715), np.float32(0.09845157), np.float32(0.14255872), np.float32(0.14531866), np.float32(0.3323421), np.float32(0.9259946), np.float32(-0.68804103), np.float32(0.00

In [9]:
print([float(x) for x in compound_scores])

[0.855858564376831, -0.9213480353355408, 0.5051683783531189, -0.2585920989513397, 0.027103319764137268, 0.02534368634223938, -0.42607229948043823, 0.6375138759613037, -0.027394693344831467, -0.060982055962085724, -0.8650408983230591, 0.05820810794830322, 0.02173355221748352, -0.02392720803618431, -0.003807060420513153, 0.07399119436740875, 0.01475205272436142, 0.15395575761795044, -0.02230854332447052, -0.02028115838766098, 0.23328599333763123, -0.8916016221046448, 0.015455309301614761, 0.1479208767414093, -0.7310732007026672, -0.9556406736373901, -0.005254596471786499, 0.042240820825099945, 0.005023345351219177, -0.01946437358856201, 0.03536549210548401, 0.755142092704773, -0.8277638554573059, -0.17578715085983276, 0.09845156967639923, 0.1425587236881256, 0.1453186571598053, 0.33234208822250366, 0.9259945750236511, -0.6880410313606262, 0.008813988417387009, -0.1765604019165039, 0.20970158278942108, 0.03354199603199959, 0.08203529566526413, -0.46145039796829224, 0.14147228002548218, 0.

In [10]:
len(compound_scores)

100

## start from scratch

**News Data**

In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

In [ ]:
def get_news_data(API_KEY: str, from_date: str, to_date:str):
    start = datetime.strptime(from_date, "%Y-%m-%d").date()
    end = datetime.strptime(to_date, "%Y-%m-%d").date()
    dates_generated = [start + timedelta(days=x) for x in range(0, (end-start).days+1)]
    print("dates done")

    articles = []
    for days in dates_generated:
        days = datetime.strftime(days, "%Y-%m-%d")

        BASE_URL = "https://content.guardianapis.com/search"
        params = {
            "api-key": API_KEY,
            "from-date": days,
            "to-date": days,
            "section": "business",
            # "sectionName": "business news",
            # "q": f"{dow_jones_companies[ticker]}",
            "show-fields": "headline, body",
            "order-by": "newest",
            "page-size": 100
        }

        response = requests.get(BASE_URL, params)
        if response is not None:
            data = response.json()

            for article in data["response"]["results"]:
                articles.append({
                    "Title": article["webTitle"],
                    "URL": article["webUrl"],
                    "Publication Date": article["webPublicationDate"],
                })
        else:
            pass

    df = pd.DataFrame(articles)
    df = df.set_index("Publication Date")

    return df

In [3]:
news = get_news_data("1e78027b-d07c-4e35-9a0a-8f1d2b4e5549", "2015-03-02", "2025-02-27")
# 1e78027b-d07c-4e35-9a0a-8f1d2b4e5549
# 75e3c8c0-28e6-4166-961c-a72883c8ea3a

dates done


KeyError: 'response'

In [9]:
news.head()

,Title,URL
Publication Date,,
2025-03-02T15:23:46Z,Chinese manufacturing returns to growth despite threat of higher Trump tariffs,https://www.theguardian.com/business/2025/mar/02/chinese-manufacturing-surges-despite-threat-of-higher-trump-tariffs
2025-03-02T15:17:25Z,Pay soars at Barclays and HSBC after end of UK banker bonus cap,https://www.theguardian.com/business/2025/mar/02/pay-soars-at-barclays-and-hsbc-after-end-of-uk-banker-bonus-cap
2025-03-02T13:00:10Z,US Postal Service faces murky future as Trump mulls dismantling institution,https://www.theguardian.com/business/2025/mar/02/usps-trump-postal-service-cuts
2025-03-02T13:00:08Z,Read the signs of Trump’s federal firings: AI is coming for private sector jobs too,https://www.theguardian.com/business/2025/mar/02/ai-layoffs-trump-irs
2025-03-02T11:11:33Z,The world has changed – it's time for a radical Labour rethink on the economy | Heather Stewart,https://www.theguardian.com/business/2025/mar/02/the-world-has-changed-its-time-for-a-radical-labour-rethink-on-the-economy


In [10]:
len(news)

34

**News Sntiment**

In [11]:
import pandas as pd
import numpy as np
import scipy
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def calculate_compound_scores(probabilities):
    return probabilities['positive'] - probabilities['negative']

def get_news_sentiment(df):

    titles = list(df["Title"].values)
    
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
    
    inputs = tokenizer(titles, return_tensors="pt", padding=True, truncation=True)
    
    with torch.no_grad():
        logits = model(**inputs).logits
    print("[Logits generated...]")
    print()
    
    probabilities = scipy.special.softmax(logits.numpy(), axis=1)
    labels = list(model.config.id2label.values())
    print("[Labels are listed...]")
    print()
    
    sentiment_scores = []
    for i, title in enumerate(titles):
        scores_dict = {labels[j]: probabilities[i][j] for j in range(len(labels))}
        sentiment_scores.append(scores_dict)
    print("[Sentiment scores are given...]")
    print()
    
    compound_scores = [calculate_compound_scores(scores) for scores in sentiment_scores]

    return compound_scores

c:\ProgramData\anaconda3\envs\RL_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
scores = get_news_sentiment(news)

[Logits generated...]

[Labels are listed...]

[Sentiment scores are given...]



In [14]:
scores = [float(x) for x in scores]
scores

[0.9259945750236511,
 -0.6880415678024292,
 -0.673867404460907,
 -0.9123600721359253,
 0.06311866641044617,
 0.5970717668533325,
 0.5299166440963745,
 0.0446418821811676,
 -0.9306768774986267,
 0.23444879055023193,
 0.586463987827301,
 0.3635013997554779,
 -0.9600926041603088,
 0.20302410423755646,
 -0.8668992519378662,
 -0.011744424700737,
 0.05846864730119705,
 -0.5932324528694153,
 -0.954470694065094,
 -0.33148813247680664,
 0.039252087473869324,
 -0.7021809816360474,
 -0.1536097526550293,
 -0.0914488285779953,
 -0.7376425862312317,
 -0.8615738749504089,
 -0.2592001259326935,
 -0.4538300037384033,
 -0.3941092789173126,
 -0.6860092878341675,
 0.11698095500469208,
 -0.9397233128547668,
 0.906383752822876,
 -0.706442654132843]

In [15]:
news["Sentiment Score"] = scores
news

,Title,URL,Sentiment Score
Publication Date,,,
2025-03-02T15:23:46Z,Chinese manufacturing returns to growth despite threat of higher Trump tariffs,https://www.theguardian.com/business/2025/mar/02/chinese-manufacturing-surges-despite-threat-of-higher-trump-tariffs,0.925995
2025-03-02T15:17:25Z,Pay soars at Barclays and HSBC after end of UK banker bonus cap,https://www.theguardian.com/business/2025/mar/02/pay-soars-at-barclays-and-hsbc-after-end-of-uk-banker-bonus-cap,-0.688042
2025-03-02T13:00:10Z,US Postal Service faces murky future as Trump mulls dismantling institution,https://www.theguardian.com/business/2025/mar/02/usps-trump-postal-service-cuts,-0.673867
2025-03-02T13:00:08Z,Read the signs of Trump’s federal firings: AI is coming for private sector jobs too,https://www.theguardian.com/business/2025/mar/02/ai-layoffs-trump-irs,-0.912360
2025-03-02T11:11:33Z,The world has changed – it's time for a radical Labour rethink on the economy | Heather Stewart,https://www.theguardian.com/business/2025/mar/02/the-world-has-changed-its-time-for-a-radical-labour-rethink-on-the-economy,0.063119
2025-03-02T07:00:03Z,"In renouncing aid and Europe, Starmer is sucking up to Trump | William Keegan",https://www.theguardian.com/business/2025/mar/02/in-renouncing-aid-and-europe-starmer-is-sucking-up-to-trump,0.597072
2025-03-03T22:00:27Z,"Soaring UK crime costing up to £250bn a year, says thinktank",https://www.theguardian.com/business/2025/mar/03/soaring-uk-crime-cost-up-policy-exchange-policing-prisons,0.529917
2025-03-03T18:33:31Z,"Jes Staley received images of ‘mature women’ from Jeffrey Epstein, court hears",https://www.theguardian.com/business/2025/mar/03/ex-barclays-ceo-jes-staley-tells-court-bank-well-aware-of-his-jeffrey-epstein-links,0.044642
2025-03-03T17:22:58Z,"UK rail passengers may lose patience as problems take years to fix, says minister",https://www.theguardian.com/business/2025/mar/03/ukrail-passengers-problems-lord-hendy-northern,-0.930677
